In [1]:
# pip install MetaTrader5

In [2]:
#Import libraries
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import pytz
from datetime import datetime
from datetime import timedelta

In [3]:
#pip install --upgrade MetaTrader5

In [4]:
import MetaTrader5 as mt5
# display data on the MetaTrader 5 package
print("MetaTrader5 package author: ",mt5.__author__)
print("MetaTrader5 package version: ",mt5.__version__)


MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.4200


In [5]:
# establish MetaTrader 5 connection to a specified trading account
if not mt5.initialize(login=10002126056, server="MetaQuotes-Demo",password="U+7eXcYp"):
    print("initialize() failed, error code =",mt5.last_error())
    quit()


In [6]:
# display data on connection status, server name and trading account
print(mt5.terminal_info())


TerminalInfo(community_account=False, community_connection=False, connected=True, dlls_allowed=False, trade_allowed=True, tradeapi_disabled=False, email_enabled=False, ftp_enabled=False, notifications_enabled=False, mqid=False, build=4260, maxbars=100000, codepage=0, ping_last=117320, community_balance=0.0, retransmission=1.118432769367765, company='MetaQuotes Software Corp.', name='MetaTrader 5', language='English', path='C:\\Program Files\\MetaTrader 5', data_path='C:\\Users\\NOC\\AppData\\Roaming\\MetaQuotes\\Terminal\\D0E8209F77C8CF37AD8BF550E51FF075', commondata_path='C:\\Users\\NOC\\AppData\\Roaming\\MetaQuotes\\Terminal\\Common')


In [7]:
# display data on MetaTrader 5 version
print(mt5.version())
 


(500, 4260, '22 Mar 2024')


In [8]:
# shut down connection to the MetaTrader 5 terminal
# mt5.shutdown()

# Connecting mt5
account = 48438229
authorized = mt5.login(account, password="Mq3p8!tJ", server="HFMarketsGlobal-Demo")
if authorized:
    account_info_dict = mt5.account_info()._asdict()
    for prop in account_info_dict:
        print(" {}={}".format(prop, account_info_dict[prop]))
else:
    print("Failed to connect at account #{}, error code: {}".format(account, mt5.last_error()))
    

In [9]:
# set time zone to UTC
timezone = pytz.timezone("Etc/UTC")


In [10]:
# select symbol
selected = mt5.symbol_select("GBPUSD", True)
if not selected:
    print("Failed to select symbol")
    mt5.shutdown()
    quit()
    

In [11]:
# create datetime object in UTC time zone to avoid the implementation of a local time zone offset
utc_from = datetime(2021, 10, 31, tzinfo = timezone)
utc_to = datetime(2022, 10, 31, tzinfo = timezone)


In [12]:
# get the bar data
rates_H4 = mt5.copy_rates_range("GBPUSD", mt5.TIMEFRAME_H4, utc_from, utc_to)


In [13]:
# shut down connection to the Metatrader 5 terminal
mt5.shutdown()


True

In [14]:
rates_H4

array([(1635724800, 1.36807, 1.36878, 1.36655, 1.36814,  8127, 0, 0),
       (1635739200, 1.36812, 1.36876, 1.36723, 1.36866,  8848, 0, 0),
       (1635753600, 1.36867, 1.36867, 1.36427, 1.36561, 21170, 0, 0), ...,
       (1666972800, 1.15388, 1.16   , 1.15335, 1.15884, 53323, 0, 0),
       (1666987200, 1.15884, 1.16233, 1.15827, 1.16141, 15877, 4, 0),
       (1667174400, 1.15858, 1.16134, 1.15795, 1.1594 , 23578, 0, 0)],
      dtype=[('time', '<i8'), ('open', '<f8'), ('high', '<f8'), ('low', '<f8'), ('close', '<f8'), ('tick_volume', '<u8'), ('spread', '<i4'), ('real_volume', '<u8')])

In [15]:
# put received rates into dataframe
df_raw = pd.DataFrame(rates_H4)

# convert time in seconds into the datetime format
df_raw['time'] = pd.to_datetime(df_raw['time'], unit = 's')


In [16]:
df_raw

,time,open,high,low,close,tick_volume,spread,real_volume
0,2021-11-01 00:00:00,1.36807,1.36878,1.36655,1.36814,8127,0,0
1,2021-11-01 04:00:00,1.36812,1.36876,1.36723,1.36866,8848,0,0
2,2021-11-01 08:00:00,1.36867,1.36867,1.36427,1.36561,21170,0,0
3,2021-11-01 12:00:00,1.36563,1.36934,1.36419,1.36735,18994,0,0
4,2021-11-01 16:00:00,1.36735,1.36926,1.36531,1.36600,20233,0,0
...,...,...,...,...,...,...,...,...
1556,2022-10-28 08:00:00,1.15722,1.15733,1.15035,1.15269,43401,1,0
1557,2022-10-28 12:00:00,1.15274,1.15751,1.15125,1.15387,47452,0,0
1558,2022-10-28 16:00:00,1.15388,1.16000,1.15335,1.15884,53323,0,0
1559,2022-10-28 20:00:00,1.15884,1.16233,1.15827,1.16141,15877,4,0


In [17]:
# count bull and bear bars
all_bars = len(df_raw.index)
bulls = 0
bears = 0
for pos, d in df_raw.iterrows():
    if(d.close > d.open):
        bulls += 1
    elif(d.close < d.open):
        bears += 1
        
print(f'All bars in given timeframe: {all_bars}')
print(f'Bull bars in given timeframe: {bulls} {bulls*100/all_bars:.0f}%')
print(f'Bear bars in given timeframe: {bears} {bears*100/all_bars:.0f}%')

All bars in given timeframe: 1561
Bull bars in given timeframe: 779 50%
Bear bars in given timeframe: 780 50%


In [ ]:
import tradingview_ta

# Set your API credentials
api_key = "YOUR_API_KEY"
api_secret = "YOUR_API_SECRET"

# Set the PineScript code
pinescript_code = """
// Your PineScript code here
"""

# Execute the PineScript code
result = tradingview_ta.execute_pinescript(
    api_key=api_key,
    api_secret=api_secret,
    script=pinescript_code
)

# Print the result
print(result)